# PyTorch CNN - Dogs vs. Cats

In [5]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [6]:
# Training Settings
TRAIN_DIR = '../data/dogs-vs-cats/train/'
VAL_DIR = '../data/dogs-vs-cats/val/'
MODEL_OUTPUT_FILEPATH = '../data/dogs-vs-cats/pytorch_model/dvc.pt'
CLASS_NAMES = ['cat', 'dog']
epochs = 15


# Define Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Size
        # Batch Size, Channels, X, Y
        #torch.Size([32, 3, 224, 224])
        
        #Input channels = 3, output channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #Input channels = 16, output channels = 32
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Keep same pooling for conv2
        
        #Input channels = 32, output channels = 64
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Keep same pooling for conv3
        self.bn1_2d = nn.BatchNorm2d(64)
        
        # Channel * X * Y
        self.fc1 = nn.Linear(64 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.bn1_1d = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, len(CLASS_NAMES))

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.bn1_2d(self.conv3(x))))
        x = x.view(-1, 64 * 28 * 28) # Reshape
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.bn1_1d(self.fc4(x)))
        x = F.log_softmax(self.fc5(x), dim=1)
        return x

net = Net()

# Define Loss and Optimization
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters())

# Image Transformations
transformations = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [7]:
# Load data into a torch dataloader
train_set = torchvision.datasets.ImageFolder(TRAIN_DIR, transform = transformations)
val_set = torchvision.datasets.ImageFolder(VAL_DIR, transform = transformations)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)

In [8]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Move model to the device specified above
net.to(device)

cuda


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1_2d): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (bn1_1d): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
best_val_accuracy = 0

for epoch in range(epochs):
    start_time = time.time()
    total_epoch_train_loss = 0
    total_epoch_val_loss = 0
    total_epoch_train_accuracy = 0
    total_epoch_val_accuracy = 0
    
    print(f'== Epoch {epoch+1} ==')
    
    # Training the model
    net.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to data to device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Clear optimizers
        optimizer.zero_grad()
        
        # Forward pass, Calc Loss, Calc Gradients, Adjust Params
        output = net.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        # Add the loss to the training set's total running loss for the epoch
        total_epoch_train_loss += loss.item()*inputs.size(0)
        
        # Get Accuracy
        # Since our model outputs a LogSoftmax, find the real 
        # percentages by reversing the log function
        output = torch.exp(output)
        # Get the top class of the output
        top_p, top_class = output.topk(1, dim=1)
        # See how many of the classes were correct?
        equals = top_class == labels.view(*top_class.shape)
        # Calculate the mean (get the accuracy for this batch)
        # and add it to the running accuracy for this epoch
        total_epoch_train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        # Print the progress of our training
        counter += 1
        print(f'Train Batches: {counter} / {len(train_loader)}', end='\r')
    print(f'Train Batches: {counter} / {len(train_loader)}')
    
    # Evaluating the model
    net.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move to data device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass, Calc Loss
            output = net.forward(inputs)
            val_loss = criterion(output, labels)
            
            # Add loss to the validation set's total running loss for the epoch
            total_epoch_val_loss += val_loss.item()*inputs.size(0)
            
            # Get Accuracy
            # Since our model outputs a LogSoftmax, find the real 
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            total_epoch_val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            # Print the progress of our evaluation
            counter += 1
            print(f'Val Batches: {counter} / {len(val_loader)}', end='\r')
    print(f'Val Batches: {counter} / {len(val_loader)}')
    
    # Get the average loss for the entire epoch
    mean_epoch_train_loss = total_epoch_train_loss / len(train_loader.dataset)
    mean_epoch_val_loss = total_epoch_val_loss / len(val_loader.dataset)
    mean_epoch_train_accuracy = total_epoch_train_accuracy / len(train_loader)
    mean_epoch_val_accuracy = total_epoch_val_accuracy / len(val_loader)
    
    # Save Model if Accuracy is highest (You can change this to whatever, instead of accuracy)
    if mean_epoch_val_accuracy > best_val_accuracy:
        print(f'UPDATE: Current epoch accuracy of {mean_epoch_val_accuracy:.6f} better than previous best of {best_val_accuracy:.6f}')
        best_val_accuracy = mean_epoch_val_accuracy
        print(f'Saving model to {MODEL_OUTPUT_FILEPATH}')
        torch.save(net.state_dict(), MODEL_OUTPUT_FILEPATH)
    
    # Print out the information
    print(f'Training Accuracy: {mean_epoch_train_accuracy:.6f}, Validation Accuracy: {mean_epoch_val_accuracy:.6f}')
    print(f'Training Loss: {mean_epoch_train_loss:.6f}, Validation Loss: {mean_epoch_val_loss:.6f}')
    print(f'Epoch finished, took {time.time() - start_time:.2f}s\n')

== Epoch 1 ==
Train Batches: 625 / 625
Val Batches: 157 / 157
UPDATE: Current epoch accuracy of 0.760549 better than previous best of 0.000000
Saving model to ../data/dogs-vs-cats/pytorch_model/dvc.pt
Training Accuracy: 0.693800, Validation Accuracy: 0.760549
Training Loss: 0.573487, Validation Loss: 0.499761
Epoch finished, took 98.64s

== Epoch 2 ==
Train Batches: 625 / 625
Val Batches: 157 / 157
Training Accuracy: 0.791450, Validation Accuracy: 0.699244
Training Loss: 0.448540, Validation Loss: 0.609236
Epoch finished, took 98.24s

== Epoch 3 ==
Train Batches: 625 / 625
Val Batches: 157 / 157
UPDATE: Current epoch accuracy of 0.803543 better than previous best of 0.760549
Saving model to ../data/dogs-vs-cats/pytorch_model/dvc.pt
Training Accuracy: 0.817900, Validation Accuracy: 0.803543
Training Loss: 0.399235, Validation Loss: 0.425123
Epoch finished, took 95.41s

== Epoch 4 ==
Train Batches: 625 / 625
Val Batches: 157 / 157
Training Accuracy: 0.842050, Validation Accuracy: 0.79279